In [ ]:
import requests
import os
import json
import time
from kafka import KafkaConsumer, KafkaProducer

producer = KafkaProducer(bootstrap_servers='localhost:9092',api_version=(0,1,0))
topic_name = 'iphone13'

bearer_token = 'AAAAAAAAAAAAAAAAAAAAAOhAAAAAdFaeYbFkEjnB%2FFEbQtpAHOI6eQg%3DneHkNg9cENkXqtXvLMP3f4fUHeUHgcDRD6Ak6REKGG20tVezt4'

search_url = "https://api.twitter.com/2/tweets/search/all"

query_params = {'query': '#AppleEvent','tweet.fields': 'created_at,author_id','expansions':'author_id','user.fields': 'name,verified,location,profile_image_url,public_metrics,description','max_results':'10'}

def bearer_oauth(r):

    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2FullArchiveSearchPython"
    return r

def connect_to_endpoint(url, params):
    response = requests.request("GET", search_url, auth=bearer_oauth, params=params)
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

def main():
    json_response = connect_to_endpoint(search_url, query_params)
    result = json_response["includes"]["users"]
    token = json_response["meta"]["next_token"]
    print(token)
    print(len(result))
    
    for user in result:
        print(user)
        producer.send(topic_name, value = json.dumps(user).encode('utf-8'))
        
    while (token != None):
        query_params2 = {'query': '#AppleEvent','next_token':token,'tweet.fields': 'created_at,author_id','expansions':'author_id','user.fields': 'name,verified,location,profile_image_url,public_metrics,description','max_results':'10'}
        json_response2 = connect_to_endpoint(search_url, query_params2)
        result2 = json_response2["includes"]["users"]
        token = json_response2["meta"]["next_token"]
        print(token)
        print(len(result2))
        for user in result:
            print(user)
            producer.send(topic_name, value = json.dumps(user).encode('utf-8'))
        time.sleep(15)
    
    #print(json.dumps(result, indent=4, sort_keys=True))
    #print(json.dumps(json_response, indent=4, sort_keys=True))

In [ ]:
if __name__ == "__main__":
    main()

200
b26v89c19zqg8o3fpz2m15nayp72xtfxrksn72v73g0ot
10
{'description': 'bio', 'verified': False, 'public_metrics': {'followers_count': 4, 'following_count': 14, 'tweet_count': 2498, 'listed_count': 0}, 'location': 'location', 'id': '2790164646', 'name': 'Ricardo Rodriguez', 'profile_image_url': 'https://abs.twimg.com/sticky/default_profile_images/default_profile_normal.png', 'username': 'rr70026'}
{'description': 'Go for the bag. Chelsea fc', 'verified': False, 'public_metrics': {'followers_count': 2383, 'following_count': 1501, 'tweet_count': 5080, 'listed_count': 0}, 'location': 'Uganda', 'id': '1492117502725431299', 'name': 'Scofie', 'profile_image_url': 'https://pbs.twimg.com/profile_images/1543099327094169600/4j_K2DRU_normal.jpg', 'username': 'scofie_felix'}
{'description': '#LifeStyle blog with #NewIdeas related to #HomeDecor, #Health #Fitness and #Traveling. Also, featuring articles on #Moms, #Parenting, #Pets,#RealEstate and more', 'verified': False, 'public_metrics': {'followers

200
b26v89c19zqg8o3fpz2m13h2p6xze6c1sgu6wt9unj8u5
9
{'description': 'bio', 'verified': False, 'public_metrics': {'followers_count': 4, 'following_count': 14, 'tweet_count': 2498, 'listed_count': 0}, 'location': 'location', 'id': '2790164646', 'name': 'Ricardo Rodriguez', 'profile_image_url': 'https://abs.twimg.com/sticky/default_profile_images/default_profile_normal.png', 'username': 'rr70026'}
{'description': 'Go for the bag. Chelsea fc', 'verified': False, 'public_metrics': {'followers_count': 2383, 'following_count': 1501, 'tweet_count': 5080, 'listed_count': 0}, 'location': 'Uganda', 'id': '1492117502725431299', 'name': 'Scofie', 'profile_image_url': 'https://pbs.twimg.com/profile_images/1543099327094169600/4j_K2DRU_normal.jpg', 'username': 'scofie_felix'}
{'description': '#LifeStyle blog with #NewIdeas related to #HomeDecor, #Health #Fitness and #Traveling. Also, featuring articles on #Moms, #Parenting, #Pets,#RealEstate and more', 'verified': False, 'public_metrics': {'followers_

KeyboardInterrupt: 

In [ ]:
producer.flush()

In [ ]:
producer.close()